In [ ]:

import os
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, NMF
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.integrate import simpson
from sklearn.metrics import jaccard_score
from nilearn import plotting
from scipy.stats import zscore


In [ ]:

# === PATHS ===
data_dir = './data'
coords_path = os.path.join(data_dir, 'MNI_66_coords.txt')
hub_metrics_path = os.path.join(data_dir, 'hub_metrics_all_subjects.mat')
fig_dir = './figures/group_consensus'
os.makedirs(fig_dir, exist_ok=True)


In [ ]:

# === Load coordinates ===
coords = pd.read_csv(coords_path, sep="\t", header=None).values[:, :3]
n_nodes = coords.shape[0]


In [ ]:

# === Load percolation matrices and compute consensus ===
file_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir)
              if f.endswith('broadband_psi_adj_participation_in_percolation.mat')]

consensus_votes = np.zeros(n_nodes)

for path in file_paths:
    mat = scipy.io.loadmat(path)
    if 'node_participation_at_percolation' not in mat:
        continue
    x = mat['node_participation_at_percolation'].T  # (nodes, steps)

    # --- Crop to longest non-NaN block ---
    x_non_nan = ~np.isnan(np.sum(x, axis=0))
    D = np.diff(np.concatenate([[0], x_non_nan.astype(int), [0]]))
    starts = np.where(D == 1)[0]
    ends = np.where(D == -1)[0]
    if len(starts) == 0 or len(ends) == 0:
        continue
    longest = np.argmax(ends - starts)
    x_crop = x[:, starts[longest]:ends[longest]]
    x_crop = np.nan_to_num(x_crop)

    # --- Flatten and weight ---
    flattening = -np.linspace(1 / x_crop.shape[1], 1, x_crop.shape[1])
    weights = np.linspace(1, 1 / x_crop.shape[1], x_crop.shape[1])
    x_flat = x_crop + flattening[np.newaxis, :]
    x_weighted = x_flat * weights[np.newaxis, :]
    x_weighted = np.clip(x_weighted, 0, None)

    # --- Cluster nodes (Ward + AUC) ---
    Z = linkage(x_weighted, method='ward')
    labels = fcluster(Z, 2, criterion='maxclust')
    auc_vals = np.array([simpson(x_weighted[i, :]) for i in range(x_weighted.shape[0])])
    crit_label = np.argmax([np.mean(auc_vals[labels == l]) for l in np.unique(labels)])
    crit_nodes = np.where(labels == crit_label + 1)[0]
    consensus_votes[crit_nodes] += 1

# === Normalize consensus votes ===
n_subs = len(file_paths)
consensus_score = consensus_votes / n_subs


In [ ]:

def binary_top_15(score):
    threshold = np.percentile(score, 85)
    return (score >= threshold).astype(int)

# === Plot maps ===
highlight_vals = consensus_score * binary_top_15(consensus_score)

plotting.plot_markers(
    node_values=highlight_vals,
    node_coords=coords,
    node_cmap='Reds',
    title='Top Nodes via Percolation Consensus',
    display_mode='lzr',
    output_file=os.path.join(fig_dir, 'percolation_consensus_top_nodes.png')
)
